In [1]:
import pandas as pd
import numpy as np

import gc
gc.collect()


0

### Load all seasons

In [2]:
seasons = list(range(2009,2019)) # creates list of years through 2018

# merge seasons into one df
df = pd.DataFrame()
for season in seasons:
    path = './../data/nflscrapR-data/play_by_play_data/regular_season/reg_pbp_'+str(season)+'.csv'
    season_df = pd.read_csv(path)
    season_df['season'] = season
    df = pd.concat([df, season_df], axis=0)
#     path = './../data/nflscrapR-data/play_by_play_data/post_season/post_pbp_'+str(season)+'.csv'
#     post_season_df = pd.read_csv(path)
#     post_season_df['season'] = season
#     df = pd.concat([df, post_season_df], axis=0)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (42,166,167,174,175,178,179,182,183,188,189,190,191,203,204,205,231,232,233,238,240,241) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (37,38,39,42,46,47,166,167,168,169,172,173,174,175,176,177,178,179,182,183,184,185,188,189,190,191,192,193,194,195,198,199,203,204,205,224,225,226,227,231,232,233,238,240,241,244,245,246,250) have mixed types. Speci

### Data Preprocessing

In [99]:
print(len(df))

# standardize jaguars abbrev
df['posteam']=df['posteam'].replace('JAC','JAX')

# drop redzone plays & inside own 5
df = df.loc[(df.yardline_100 > 20) & (df.yardline_100 < 95)]

# drop extreme win probabilities (about 4% of plays)
df = df.loc[(df['wp']>=0.05) | (df['wp']<=0.95)]

# drop afc and nfc

conf_teams = ['AFC','NFC','IRV','APR','NPR','RIC','SANS']
df = df.loc[~df['posteam'].isin(conf_teams)]
print(len(df))



362283
362283


### Load Roster Data

found here: https://github.com/btatkinson/pfr_scraper

In [102]:
path = './../data/roster/roster_info.csv'
roster = pd.read_csv(path)
print(roster.head())
roster['sea_id'] = roster['season'].astype(str)+roster['player_id']

   season posteam                       id   age    av  \
0    2009     PIT  2009PITB.Roethlisberger  27.0  14.0   
1    2009     TEN         2009TENK.Collins  37.0   2.0   
2    2009     CLE           2009CLEB.Quinn  25.0   2.0   
3    2009     MIN           2009MINB.Favre  40.0  16.0   
4    2009      NO            2009NOD.Brees  30.0  16.0   

                                      draft   exp  height                name  \
0  Pittsburgh Steelers / 1st / 11th pick /    5.0    77.0  Ben Roethlisberger   
1     Carolina Panthers / 1st / 5th pick /   14.0    77.0       Kerry Collins   
2     Cleveland Browns / 1st / 22nd pick /    2.0    76.0         Brady Quinn   
3      Atlanta Falcons / 2nd / 33rd pick /   18.0    74.0         Brett Favre   
4   San Diego Chargers / 2nd / 32nd pick /    8.0    72.0          Drew Brees   

  pos salary        short_name team  weight   player_id  
0  QB    NaN  B.Roethlisberger  PIT   240.0  00-0022924  
1  QB    NaN         K.Collins  TEN   247.0  00-

### QB heatmap

In [104]:
qbs = df.copy()

# drop unnecessary columns
qbs = qbs.drop(columns=['assist_tackle', 'assist_tackle_1_player_id', 'assist_tackle_1_player_name', 'assist_tackle_1_team', 'assist_tackle_2_player_id', 'assist_tackle_2_player_name', 'assist_tackle_2_team', 'assist_tackle_3_player_id', 'assist_tackle_3_player_name', 'assist_tackle_3_team', 'assist_tackle_4_player_id', 'assist_tackle_4_player_name', 'assist_tackle_4_team',
                       'kick_distance', 'kicker_player_id', 'kicker_player_name', 'kickoff_attempt', 'kickoff_downed', 'kickoff_fair_catch', 'kickoff_in_endzone', 'kickoff_inside_twenty', 'kickoff_out_of_bounds', 'kickoff_returner_player_id', 'kickoff_returner_player_name', 'lateral_interception_player_id', 'lateral_interception_player_name', 'lateral_kickoff_returner_player_id', 
                        'lateral_kickoff_returner_player_name', 'lateral_punt_returner_player_id', 'lateral_punt_returner_player_name', 'lateral_receiver_player_id', 'lateral_receiver_player_name', 'lateral_reception', 'lateral_recovery', 'lateral_return', 'lateral_rush', 'lateral_rusher_player_id', 'lateral_rusher_player_name', 'lateral_sack_player_id', 'lateral_sack_player_name',
                         'own_kickoff_recovery','own_kickoff_recovery_player_id','own_kickoff_recovery_player_name','own_kickoff_recovery_td','solo_tackle', 'solo_tackle_1_player_id', 'solo_tackle_1_player_name', 'solo_tackle_1_team', 'solo_tackle_2_player_id', 'solo_tackle_2_player_name', 'solo_tackle_2_team','punt_attempt', 'punt_blocked', 'punt_downed', 'punt_fair_catch', 'punt_in_endzone', 
                        'punt_inside_twenty', 'punt_out_of_bounds', 'punt_returner_player_id', 'punt_returner_player_name', 'punter_player_id', 'punter_player_name','total_away_comp_air_epa', 'total_away_comp_air_wpa', 'total_away_comp_yac_epa', 'total_away_comp_yac_wpa', 'total_away_epa', 'total_away_pass_epa', 'total_away_pass_wpa', 'total_away_raw_air_epa', 'total_away_raw_air_wpa', 'total_away_raw_yac_epa', 
                        'total_away_raw_yac_wpa', 'total_away_rush_epa', 'total_away_rush_wpa', 'total_away_score', 'total_home_comp_air_epa', 'total_home_comp_air_wpa', 'total_home_comp_yac_epa', 'total_home_comp_yac_wpa', 'total_home_epa', 'total_home_pass_epa', 'total_home_pass_wpa', 'total_home_raw_air_epa', 'total_home_raw_air_wpa', 'total_home_raw_yac_epa', 'total_home_raw_yac_wpa', 'total_home_rush_epa', 
                        'total_home_rush_wpa', 'total_home_score', 'touchback', 'two_point_attempt', 'two_point_conv_result'])

qb_ids = roster.loc[roster.pos=='QB']
qb_ids = list(qb_ids.player_id.values)

print(len(qbs))
qbs = qbs.loc[(qbs['passer_player_id'].isin(qb_ids)) | qbs['rusher_player_id'].isin(qb_ids)]
print(len(qbs))


qbs['season'] = qbs['season'].astype(str)

qb_rushes = qbs.loc[qbs.play_type=='run']
qb_passes = qbs.loc[qbs.play_type=='pass']

qb_rushes['id'] = qb_rushes['season'] + qb_rushes['posteam'] + qb_rushes['rusher_player_name']
qb_passes['id'] = qb_passes['season'] + qb_passes['posteam'] + qb_passes['passer_player_name']





362283
175442


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(9731, 179)
(161204, 179)


In [105]:
# only take QBs that matched roster ids. About 94% of pass plays
qb_passes = qb_passes.loc[qb_passes['player_id']==qb_passes['passer_player_id']]
qb_rushes = qb_rushes.loc[qb_rushes['player_id']==qb_rushes['rusher_player_id']]
print(qb_passes.shape)
print(qb_rushes.shape)

(151749, 179)
(9250, 179)


In [106]:
qb_passes['sea_id'] = qb_passes['season_x'].astype(str) + qb_passes['passer_player_id']
qb_rushes['sea_id'] = qb_rushes['season_x'].astype(str) + qb_rushes['rusher_player_id']

In [107]:
# only take qbs with at least 3 rushes and 100 passes
gb = qb_rushes.groupby(['sea_id'])['epa'].count()
gb = gb.loc[gb >= 3].reset_index()
run_ids = list(gb['sea_id'].unique())

gb = qb_passes.groupby(['sea_id'])['epa'].count()
gb = gb.loc[gb >= 100].reset_index()
pass_ids = list(gb['sea_id'].unique())

qb_ids = []
for pid in pass_ids:
    if pid in run_ids:
        qb_ids.append(pid)

print(str(len(qb_ids)) + " Quarterbacks meet the criteria")

388 Quarterbacks meet the criteria


In [120]:
def explosiveness(x):
    # (Warning: O(n**2) in time and memory, where n = len(x).  *Don't* pass in huge samples!)
    
    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    mean = np.mean(x)
    
    if mean == 0:
        return 0
    rmad = mad/mean
    # Gini coefficient
    g = 0.5 * rmad
    return g

In [109]:
# get game count
gc = qb_passes.groupby(['sea_id'])['game_id'].nunique()

# only use qbs with over 5 games

gc = gc.loc[gc > 5]
gc = gc.reset_index()
print(gc.head())

qb_passes = qb_passes.loc[qb_passes['sea_id'].isin(list(gc.sea_id.values))]
qb_rushes = qb_rushes.loc[qb_rushes['sea_id'].isin(list(gc.sea_id.values))]

           sea_id  game_id
0  200900-0003292        7
1  200900-0003739        8
2  200900-0004161       11
3  200900-0005106       16
4  200900-0007091       14


In [110]:
# yards per rush
ypc = qb_rushes.groupby(['sea_id'])['yards_gained'].mean().reset_index()
ypc = ypc.rename(columns={'yards_gained':'YPC'})
print(ypc.tail(5))

# win probability added on rushes (will do per game later)
wpa = qb_rushes.groupby(['sea_id'])['wpa'].sum().reset_index()
print(wpa.tail(5))

# attempts (will do per game later)
att = qb_rushes.groupby(['sea_id'])['yards_gained'].count().reset_index()
att = att.rename(columns={'yards_gained':'Att'})
print(att.tail(5))

# run explosiveness
rexp = qb_rushes.groupby(['sea_id'])['yards_gained'].apply(explosiveness).reset_index()
rexp = rexp.rename(columns={'yards_gained':'Run_Exp'})
print(rexp.tail(5))

             sea_id       YPC
367  201800-0034343  9.062500
368  201800-0034796  5.910891
369  201800-0034855  5.904762
370  201800-0034857  8.741935
371  201800-0034869  4.961538
             sea_id       wpa
367  201800-0034343  0.100573
368  201800-0034796  0.405059
369  201800-0034855  0.292732
370  201800-0034857  1.094757
371  201800-0034869 -0.057479
             sea_id  Att
367  201800-0034343   16
368  201800-0034796  101
369  201800-0034855   21
370  201800-0034857   62
371  201800-0034869   26
             sea_id   Run_Exp
367  201800-0034343  0.471983
368  201800-0034796  0.580560
369  201800-0034855  0.640553
370  201800-0034857  0.543983
371  201800-0034869  0.547108


In [111]:
# merge in stats
qb_run_dfs = [ypc,wpa,att,rexp]
for rdf in qb_run_dfs:
    gc = pd.merge(how='left',left=gc,right=rdf,on=['sea_id'])

In [112]:
gc['Run Att/G'] = gc['Att']/gc['game_id']
gc['Run WPA/G'] = gc['wpa']/gc['game_id']

gc = gc.drop(columns=['Att','wpa'])

In [113]:
print(list(qb_passes))

['air_epa', 'air_wpa', 'air_yards', 'away_team', 'away_timeouts_remaining', 'away_wp', 'away_wp_post', 'blocked_player_id', 'blocked_player_name', 'comp_air_epa', 'comp_air_wpa', 'comp_yac_epa', 'comp_yac_wpa', 'complete_pass', 'def_wp', 'defensive_extra_point_attempt', 'defensive_extra_point_conv', 'defensive_two_point_attempt', 'defensive_two_point_conv', 'defteam', 'defteam_score', 'defteam_score_post', 'defteam_timeouts_remaining', 'desc', 'down', 'drive', 'ep', 'epa', 'extra_point_attempt', 'extra_point_prob', 'extra_point_result', 'fg_prob', 'field_goal_attempt', 'field_goal_result', 'first_down_pass', 'first_down_penalty', 'first_down_rush', 'forced_fumble_player_1_player_id', 'forced_fumble_player_1_player_name', 'forced_fumble_player_1_team', 'forced_fumble_player_2_player_id', 'forced_fumble_player_2_player_name', 'forced_fumble_player_2_team', 'fourth_down_converted', 'fourth_down_failed', 'fumble', 'fumble_forced', 'fumble_lost', 'fumble_not_forced', 'fumble_out_of_bounds',

In [114]:
def is_success(epa):
    return 1 if epa >= 0 else 0

# add success column
qb_passes['success'] = qb_passes.apply(lambda row: is_success(row['epa']),axis=1)

In [128]:
# total dropbacks
dbs = qb_passes.groupby(['sea_id'])['epa'].count().reset_index()
dbs = dbs.rename(columns={'epa':'dropbacks'})

# average columns to get per dropback numbers
means = qb_passes.groupby(['sea_id'])['touchdown','interception','complete_pass','qb_hit','sack','success','epa','wp'].mean()

means = means.rename(columns={'touchdown':'TD/D','interception':'INT/D','complete_pass':'Comp/D','qb_hit':'Hit/D',
                              'sack':'Sack/D','success':'SR/D','epa':'EPA/D','wp':'Avg_WP'})
means.tail(15)



,TD/D,INT/D,Comp/D,Hit/D,Sack/D,SR/D,EPA/DB,Avg_WP
sea_id,,,,,,,,
201800-0031503,0.017241,0.031609,0.609195,0.149425,0.068966,0.500000,0.063832,0.443823
201800-0032268,0.012579,0.022013,0.628931,0.160377,0.119497,0.474843,0.032629,0.455222
201800-0032436,0.011834,0.011834,0.573964,0.165680,0.082840,0.372781,-0.104464,0.331076
201800-0032950,0.010899,0.016349,0.664850,0.174387,0.073569,0.509537,0.090963,0.477000
201800-0033077,0.022044,0.012024,0.635271,0.176353,0.094188,0.466934,0.078369,0.493865
201800-0033106,0.020704,0.020704,0.627329,0.118012,0.057971,0.538302,0.207121,0.546386
201800-0033319,0.024590,0.028689,0.618852,0.229508,0.057377,0.495902,0.181702,0.414823
201800-0033537,0.012848,0.019272,0.642398,0.216274,0.100642,0.498929,0.090199,0.543441
201800-0033869,0.015464,0.028351,0.636598,0.095361,0.054124,0.481959,0.067629,0.622450


In [122]:
# divide into completions and incompletions
qb_comps = qb_passes.loc[qb_passes.complete_pass==1]
qb_incomps = qb_passes.loc[(qb_passes.complete_pass==0) & (qb_passes.sack==0)]
qb_sacks = qb_passes.loc[(qb_passes.complete_pass==0) & (qb_passes.sack==1)]

pass_avgs = qb_comps.groupby(['sea_id'])['yards_gained','yards_after_catch','air_yards'].mean().reset_index()

# have to calc expl separately
expl_compl = qb_comps.groupby(['sea_id'])['yards_gained'].apply(explosiveness).reset_index()
expl_yac = qb_comps.groupby(['sea_id'])['yards_after_catch'].apply(explosiveness).reset_index()
expl_ay = qb_comps.groupby(['sea_id'])['air_yards'].apply(explosiveness).reset_index()

icay = qb_incomps.groupby(['sea_id'])['air_yards'].mean().reset_index

